In [78]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher
from tqdm import tqdm

In [2]:
with open('data/data_concerts.csv', encoding='utf-8') as concerts_file:
    concerts_df = pd.read_csv(concerts_file, index_col=0)
    
concerts_df

C:\Users\Cyrille\Desktop\venv\redux\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,type,name,venue,locality,venue_id,startDate,endDate,artist_1,artist_url_1,artist_2,...,artist_64,artist_url_64,artist_65,artist_url_65,artist_66,artist_url_66,artist_67,artist_url_67,artist_68,artist_url_68
/festivals/3376423,festival,EDM Zinne w/Nehilo & Alex Napster 2021,"Sargans, Switzerland",Sargans,/venues/1943879,2021-08-14,2021-08-15,Flavio Stonex,/artists/10069604-flavio-stonex,Nehilo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/festivals/3370309,festival,P20 - EDM Night Zinne Sargans 2021,"Sargans, Switzerland",Sargans,/venues/1943879,2021-07-02,2021-07-03,Flavio Stonex,/artists/10069604-flavio-stonex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/festivals/3338983,festival,EDM Night Vol.3 2020,"Sargans, Switzerland",Sargans,/venues/1943879,2020-10-23,2020-10-23,Flavio Stonex,/artists/10069604-flavio-stonex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/festivals/3250141,festival,Privatevent (gebucht) 2020,VHS Flums,Sargans,/venues/2077709,2020-10-17,2020-10-17,Flavio Stonex,/artists/10069604-flavio-stonex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/festivals/3326252,festival,PRESS N Play Daydance Sennwald 2020,"Sargans, Switzerland",Sargans,/venues/1943879,2020-06-20,2020-06-20,Flava & Stevenson,/artists/9379654-flava-and-stevenson,Flavio Stonex,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/concerts/14694789,concert,Demented Are Go,Komplex Klub,Zürich,/venues/2148724,2013-04-11,2013-04-11,Demented Are Go,/artists/325350-demented-are-go,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/concerts/14836239,concert,For the Fallen Dreams,Komplex Klub,Zürich,/venues/2148724,2013-04-09,2013-04-09,For the Fallen Dreams,/artists/1791519-for-the-fallen-dreams,Abandon All Ships,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/concerts/14836234,concert,Silverstein,Komplex Klub,Zürich,/venues/2148724,2013-04-07,2013-04-07,Silverstein,/artists/424864-silverstein,Funeral for a Friend,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/concerts/16089124,concert,Vatsgoed,Komplex Klub,Zürich,/venues/2148724,2013-04-05,2013-04-05,Vatsgoed,/artists/6316464-vatsgoed,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
with open('data/data_venues.csv', encoding='utf-8') as venues_file:
    venues_df = pd.read_csv(venues_file, index_col=0)
    
venues_df

,venue,locality,postal_code,street,country,latitude,longitude,venue_website
/venues/1943879,"Sargans, Switzerland",Mels,8887,NaN,Switzerland,47.04380,9.42400,NaN
/venues/2958443,"Kiental, Switzerland",Frutigen,3723,NaN,Switzerland,46.54310,7.75573,http://naturalsound.ch
/venues/78549,"Wetzikon, Switzerland",Wetzikon,NaN,NaN,Switzerland,47.32706,8.80136,NaN
/venues/29550,Hallenstadion,Zürich,8050,Wallisellenstrasse 45,Switzerland,47.41161,8.55166,http://www.hallenstadion.ch/
/venues/32239,Hive,Zürich,8005,Geroldstrasse 5,Switzerland,47.38537,8.52047,http://www.hiveclub.ch/
...,...,...,...,...,...,...,...,...
/venues/2763668,House Show,Biel,NaN,NaN,Switzerland,47.14019,7.24377,NaN
/venues/4402932,YouTube Live,Geneva,NaN,NaN,Switzerland,NaN,NaN,NaN
/venues/3754699,Mcgees Irish Pub,Zürich,8003,Birmensdorferstrasse 83,Switzerland,47.37078,8.52391,NaN
/venues/4420939,Freie Evangelische Gemeinde Emmen,Luzern,6032,Mooshüslistrasse 32,Switzerland,47.07869,8.29039,http://www.feg-emmen.ch


In [4]:
with open('data/data_artists.csv', encoding='utf-8') as artists_file:
    artists_df = pd.read_csv(artists_file, index_col=0)
    
artists_df

,artist_name
/artists/3684861-chk-chk-chk,!!! (Chk Chk Chk)
/artists/694091-kadebostan,!!!Kadebostan!!!
/artists/5929284-brothers-in-bamako,"""Brothers In Bamako"""
/artists/8704378-floyd-division-the-austrian-pink-floyd-tribute-band,"""Floyd Division"" The Austrian Pink Floyd Tribu..."
/artists/7172094-jazzetcetera-stewy-von-wattenwyl-group-feat-lisette-spinnler,"""Jazz...etcetera"" Stewy von Wattenwyl Group, ..."
...,...
/artists/4301-and-you-will-know-us-by-the-trail-of-dead,…And You Will Know Us by the Trail of Dead
/artists/1099500-euuroshima,€urOshima
/artists/9573874-belize-it,≈ Belize ≈ (IT)
/artists/8889739-luo-chai-cao-yuan-wwww-prairie-wwww,落差草原 WWWW / Prairie WWWW


In [5]:
def dict_from_df(data_df):
    data_dict = data_df.to_dict(orient='index')
    print(len(data_dict))

    for index, item_data in data_dict.items():
        for data_name, data_value in item_data.copy().items():  # using copy() because dictionary size is changed during iteration
            if pd.isna(data_value):
                item_data.pop(data_name, None)
                
    print(len(data_dict))
    return data_dict

In [6]:
concerts_dict = dict_from_df(concerts_df)
venues_dict = dict_from_df(venues_df)
artists_dict = dict_from_df(artists_df)

82011
82011
7657
7657
32231
32231


---
## Preprocess data

In [ ]:
# get the similarity between two strings

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [49]:
# find artists name that are similar to check duplicate artists

counter = 0
similar_artists_dict = dict()

for index_i, artist_data_i in tqdm(artists_dict.items()):
    counter_similarity = 1
    artist_name_i = artist_data_i['artist_name']
    similar_artists_dict[index_i] = {'artist_name': artist_name_i}
    
    for index_j, artist_data_j in artists_dict.items():
        artist_name_j = artist_data_j['artist_name']
        similarity = similar(artist_name_i, artist_name_j)
        
        if 0.75 < similarity < 1:
            similar_artists_dict[index_i]['artist_name'] = artist_name_i
            similar_artists_dict[index_i]['similar_artist_' + str(counter_similarity)] = artist_name_j
            similar_artists_dict[index_i]['similar_index_' + str(counter_similarity)] = index_j
            similar_artists_dict[index_i]['similarity_' + str(counter_similarity)] = similarity
            counter_similarity += 1
        
    counter += 1
    if counter > 200:
        break
        
len(similar_artists_dict)

  1%|▊                                                                                                                                           | 200/32231 [03:03<8:11:01,  1.09it/s]


{'/artists/3684861-chk-chk-chk': {'artist_name': '!!! (Chk Chk Chk)'},
 '/artists/694091-kadebostan': {'artist_name': '!!!Kadebostan!!!'},
 '/artists/5929284-brothers-in-bamako': {'artist_name': '"Brothers In Bamako"'},
 '/artists/8704378-floyd-division-the-austrian-pink-floyd-tribute-band': {'artist_name': '"Floyd Division" The Austrian Pink Floyd Tribute Band'},
 '/artists/7172094-jazzetcetera-stewy-von-wattenwyl-group-feat-lisette-spinnler': {'artist_name': '"Jazz...etcetera" Stewy von Wattenwyl Group,  feat. Lisette Spinnler'},
 '/artists/7433784-pyramids-jazz': {'artist_name': '"The Pyramids" (Jazz)'},
 '/artists/8375378-agcrew': {'artist_name': '#AGCrew'},
 '/artists/44425-smoove': {'artist_name': '$MOOVE'},
 '/artists/7809989-and-francis-fruit': {'artist_name': '& Francis Fruit'},
 '/artists/8672184-and-friends': {'artist_name': '& Friends',
  'similar_artist_1': 'Dodo & Friends',
  'similar_index_1': '/artists/9376779-dodo-and-friends',
  'similarity_1': 0.782608695652174,
  's

## Process Data

In [86]:
def get_filtered_concerts(concerts_df, filter_item):
    filtered_concerts_df = concerts_df[concerts_df.eq(filter_item).any(1)]
    return filtered_concerts_df

In [73]:
def get_artists_in_concerts(concerts_df):
    columns_list = ['artist_url_' + str(x) for x in range(1, 69)]
    artists_in_concerts_df = concerts_df[concerts_df.columns.intersection(columns_list)]
    
    return artists_in_concerts_df

In [88]:
def get_artists_by_venue(concerts_df, venue_id):
    concerts_in_venue_df = get_filtered_concerts(concerts_df, venue_id)
    artists_in_concerts_df = get_artists_in_concerts(concerts_in_venue_df)
    artists_in_venue_list = list(np.unique(artists_in_concerts_df.to_numpy(na_value='')))
    
    try:
        artists_in_venue_list.remove('')
    except ValueError:
        pass
    
    print(len(artists_in_venue_list))
    artists_in_venue_df = artists_df.loc[artists_in_venue_list]
    
    return artists_in_venue_df

In [87]:
# get all the concerts by a given artist

artist_id = '/artists/3684861-chk-chk-chk'
get_filtered_concerts(concerts_df, artist_id)

,type,name,venue,locality,venue_id,startDate,endDate,artist_1,artist_url_1,artist_2,...,artist_64,artist_url_64,artist_65,artist_url_65,artist_66,artist_url_66,artist_67,artist_url_67,artist_68,artist_url_68
/concerts/17832864,concert,!!! (Chk Chk Chk),Kaserne Basel,Basel,/venues/814881,2013-10-30,2013-10-30,!!! (Chk Chk Chk),/artists/3684861-chk-chk-chk,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/concerts/6255616,concert,!!! (Chk Chk Chk),Les Docks,Lausanne,/venues/35054,2010-11-17,2010-11-17,!!! (Chk Chk Chk),/artists/3684861-chk-chk-chk,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/concerts/834645,concert,!!! (Chk Chk Chk),Les Docks,Lausanne,/venues/35054,2007-10-27,2007-10-27,!!! (Chk Chk Chk),/artists/3684861-chk-chk-chk,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/concerts/834647,concert,!!! (Chk Chk Chk),Abart,Zürich,/venues/32250,2007-10-31,2007-10-31,!!! (Chk Chk Chk),/artists/3684861-chk-chk-chk,Holy Fuck,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/festivals/14646,festival,Paléo Festival 2007,Plaine de l'Asse,Nyon,/venues/926036,2007-07-24,2007-07-29,Arctic Monkeys,/artists/520117-arctic-monkeys,Muse,...,Catcha,/artists/1197953-catcha,The Mondrians,/artists/606601-mondrians,Naast,/artists/542527-naast,Inna Crisis,/artists/238866-inna-crisis,SOLARTONES,/artists/778095-solartones
/festivals/1069,festival,For Noise Festival 2007,For Noise Festival,Pully,/venues/612901,2007-08-09,2007-08-11,M83,/artists/525418-m83,Just Jack,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
# get all concerts in a given venue

venue_id = '/venues/35054'
get_filtered_concerts(concerts_df, venue_id)

,type,name,venue,locality,venue_id,startDate,endDate,artist_1,artist_url_1,artist_2,...,artist_64,artist_url_64,artist_65,artist_url_65,artist_66,artist_url_66,artist_67,artist_url_67,artist_68,artist_url_68
/festivals/1627,festival,Persistence Tour 2019,Les Docks,Lausanne,/venues/35054,2019-01-23,2019-01-23,Sick of It All,/artists/368873-sick-of-it-all,Municipal Waste,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/concerts/39337883,concert,Apparat,Les Docks,Lausanne,/venues/35054,2021-09-11,2021-09-11,Apparat,/artists/396628-apparat,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/concerts/39805831,concert,Yugen Blakrok,Les Docks,Lausanne,/venues/35054,2021-09-10,2021-09-10,Yugen Blakrok,/artists/9879829-yugen-blakrok,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/concerts/39729255,concert,Tafta,Les Docks,Lausanne,/venues/35054,2021-09-04,2021-09-04,Tafta,/artists/100487-tafta,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/concerts/39697712,concert,Rone,Les Docks,Lausanne,/venues/35054,2021-09-03,2021-09-03,Rone,/artists/490658-rone,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/concerts/10958193,concert,Gotthard,Les Docks,Lausanne,/venues/35054,2007-05-10,2007-05-10,Gotthard,/artists/447949-gotthard,Redeem,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/concerts/11029523,concert,Les Cowboys Fringants,Les Docks,Lausanne,/venues/35054,2007-04-30,2007-04-30,Les Cowboys Fringants,/artists/320186-les-cowboys-fringants,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/concerts/3154026,concert,Archive,Les Docks,Lausanne,/venues/35054,2007-04-09,2007-04-09,Archive,/artists/352738-archive,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/concerts/2378151,concert,Charlélie Couture,Les Docks,Lausanne,/venues/35054,2007-02-07,2007-02-07,Charlélie Couture,/artists/495667-charlelie-couture,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
# get all artists who played in a given venue

venue_id = '/venues/35054'
get_artists_by_venue(concerts_df, venue_id)

912


,artist_name
/artists/10004739-lepee,L'Épée
/artists/100389-rachid-taha,Rachid Taha
/artists/100487-tafta,Tafta
/artists/100494-fauve,Fauve
/artists/10058637-kallemi,Kallemi
...,...
/artists/99189-dionysos,Dionysos
/artists/99207-rinocerose,Rinôçérôse
/artists/992521-redback,REdbACK
/artists/99621-john-butler-trio,John Butler Trio
